In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('max_seq_item', None)

In [3]:
df_sen_pro_act = pd.read_csv('senators_signed_pro_act.csv')
df_amazon = pd.read_csv('amazon_donations.csv')

In [5]:
pip install fuzzy-pandas

Note: you may need to restart the kernel to use updated packages.


In [6]:
import fuzzy_pandas as fpd

In [7]:
df_sen_pro_act = df_sen_pro_act.fillna('')
df_amazon = df_amazon.fillna('')

In [8]:
df_sen_pro_act.head()

,senators
0,"Sen. Murray, Patricia [D-WA]"
1,"Sen. Schumer, Charles E. [D-NY]*"
2,"Sen. Blumenthal, Richard [D-CT]*"
3,"Sen. Duckworth, Tammy [D-IL]*"
4,"Sen. Booker, Cory A. [D-NJ]*"


In [14]:
df_sen_pro_act['last_name'] = df_sen_pro_act.senators.str.split('Sen. ', expand=True)[1].str.split(',', expand=True)[0].str.upper()
df_sen_pro_act['first_name'] = df_sen_pro_act.senators.str.split('Sen. ', expand=True)[1].str.split(',', expand=True)[1].str.split('[',expand=True)[0].str.upper()
df_sen_pro_act['first_name'] = df_sen_pro_act['first_name'].apply(lambda x: x.split()[0])
df_sen_pro_act['senator_name'] = df_sen_pro_act['last_name'].str.cat(df_sen_pro_act['first_name'], sep=' ')
df_sen_pro_act.head()

,senators,last_name,first_name,senator_name
0,"Sen. Murray, Patricia [D-WA]",MURRAY,PATRICIA,MURRAY PATRICIA
1,"Sen. Schumer, Charles E. [D-NY]*",SCHUMER,CHARLES,SCHUMER CHARLES
2,"Sen. Blumenthal, Richard [D-CT]*",BLUMENTHAL,RICHARD,BLUMENTHAL RICHARD
3,"Sen. Duckworth, Tammy [D-IL]*",DUCKWORTH,TAMMY,DUCKWORTH TAMMY
4,"Sen. Booker, Cory A. [D-NJ]*",BOOKER,CORY,BOOKER CORY


In [15]:
df_amazon.head(5)

,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,entity_type,entity_type_desc,unused_recipient_committee_id,recipient_committee_id,recipient_name,recipient_state,beneficiary_committee_name,national_committee_nonfederal_account,disbursement_type,disbursement_type_description,disbursement_description,memo_code,memo_code_full,disbursement_date,disbursement_amount,candidate_office,candidate_office_description,candidate_office_district,candidate_id,candidate_name,candidate_first_name,candidate_last_name,candidate_middle_name,candidate_prefix,candidate_suffix,candidate_office_state,candidate_office_state_full,election_type,election_type_full,fec_election_type_desc,fec_election_year,amendment_indicator,amendment_indicator_desc,schedule_type_full,load_date,original_sub_id,back_reference_transaction_id,back_reference_schedule_id,semi_annual_bundled_refund,payee_last_name,payee_first_name,payee_middle_name,category_code,category_code_full,conduit_committee_name,conduit_committee_street1,conduit_committee_street2,conduit_committee_city,conduit_committee_state,conduit_committee_zip,spender_committee_type,spender_committee_org_type,spender_committee_designation,filing_form,link_id,recipient_city,recipient_zip,disbursement_purpose_category,memo_text,two_year_transaction_period,schedule_type,sub_id,pdf_url,line_number_label,payee_prefix,payee_suffix,payee_employer,payee_occupation,ref_disp_excess_flg,comm_dt
0,C00360354,AMAZON.COM SERVICES LLC SEPARATE SEGREGATED FUND (AMAZON PAC),2022,30G,202212089557189117,23,75C537A6E9352BCC1B6,1671411,CCM,CAMPAIGN COMMITTEE,C00289603,C00289603,LOFGREN FOR CONGRESS,CA,,,24K,CONTRIBUTION MADE TO NON-AFFILIATED,VOIDED 8/8/22 DISBURSEMENT,,,2022-10-31 0:00:00,-5000.0,H,HOUSE,18.0,H4CA16049,"LOFGREN, ZOE",ZOE,LOFGREN,,,,CA,CALIFORNIA,G2022,,GENERAL,2022.0,A,ADD,ITEMIZED DISBURSEMENTS,2023-01-18 6:21:38,,,,0,,,,11,Political Contributions,,,,,,,Q,C,B,F3X,4120920221633755305,SAN JOSE,95126,CONTRIBUTIONS,,2022,SB,4011720231699468279,https://docquery.fec.gov/cgi-bin/fecimg/?202212089557189117,Contributions to Federal Candidates/Committees and Other Political Committees,,,,,,
1,C00360354,AMAZON.COM SERVICES LLC SEPARATE SEGREGATED FUND (AMAZON PAC),2021,YE,202201319475613393,23,FD3B74A7F738DB96897,1562724,CCM,CAMPAIGN COMMITTEE,C00670547,C00670547,ARMSTRONG FOR CONGRESS,ND,,,24K,CONTRIBUTION MADE TO NON-AFFILIATED,VOIDED 10/19/20 DISBURSEMENT,,,2021-11-30 0:00:00,-5000.0,H,HOUSE,1.0,H8ND00096,"ARMSTRONG, KELLY M.",KELLY,ARMSTRONG,M.,,,ND,NORTH DAKOTA,G2020,,GENERAL,2020.0,A,ADD,ITEMIZED DISBURSEMENTS,2022-02-03 3:34:08,,,,0,,,,11,Political Contributions,,,,,,,Q,C,B,F3X,4013120221403740388,BISMARCK,58503,CONTRIBUTIONS,,2022,SB,4020220221404097787,https://docquery.fec.gov/cgi-bin/fecimg/?202201319475613393,Contributions to Federal Candidates/Committees and Other Political Committees,,,,,,
2,C00360354,AMAZON.COM SERVICES LLC SEPARATE SEGREGATED FUND (AMAZON PAC),2021,YE,202201319475613447,23,565E693BAE7D9515B7B,1562724,PAC,POLITICAL ACTION COMMITTEE,C00459925,C00459925,MURPHPAC,DC,MURPHPAC,,24K,CONTRIBUTION MADE TO NON-AFFILIATED,VOIDED 12/17/20 DISBURSEMENT,,,2021-11-30 0:00:00,-5000.0,,,,,,,,,,,,,P2020,CONTRIBUTION,PRIMARY,2020.0,A,ADD,ITEMIZED DISBURSEMENTS,2022-02-03 3:34:14,,,,0,,,,11,Political Contributions,,,,,,,Q,C,B,F3X,4013120221403740388,WASHINGTON,20003,CONTRIBUTIONS,,2022,SB,4020220221404097949,https://docquery.fec.gov/cgi-bin/fecimg/?202201319475613447,Contributions to Federal Candidates/Committees and Other Political Committees,,,,,,
3,C00360354,AMAZON.COM SERVICES LLC SEPARATE SEGREGATED FUND (AMAZON PAC),2022,M8,202208119525263330,23,AFFC0AE538597C902A1,1623070,PAC,POLITICAL ACTION COMMITTEE,C00631051,C00631051,ABRAHAM LINCOLN PAC,GA,ABRAHAM LINCOLN PAC,,24K,CONTRIBUTION MADE TO NON-AFFILIATED,VOIDED 12/6/21 DISBURSEMENT,,,2022-07-11 0:00:00,-5000.0,,,,,,,,,,,,,P2021,CONTRIBUTION,PRIMARY,2021.0,A,ADD,ITEMIZED DISBURSEMENTS,2022-08-13 3:34:27,,,,0

In [17]:
amazon_sen_results = fpd.fuzzy_merge(df_amazon, df_sen_pro_act,
            left_on=['candidate_name'],
            right_on=['senators'],
            ignore_case=True,
            ignore_nonalpha=True,
            ignore_nonlatin=True,
            ignore_order_words=True,
            ignore_titles=True,
            method='levenshtein',
            threshold=0.50)

print("Found", amazon_sen_results.shape)
amazon_sen_results[['disbursement_amount', 'candidate_name', 'candidate_last_name', 'candidate_first_name', 'candidate_office', 'senators','last_name','first_name']]

Found (37, 82)


,disbursement_amount,candidate_name,candidate_last_name,candidate_first_name,candidate_office,senators,last_name,first_name
0,-2500.0,"SHAHEEN, JEANNE",SHAHEEN,JEANNE,S,"Sen. Shaheen, Jeanne [D-NH]*",SHAHEEN,JEANNE
1,-2500.0,"SMITH, CHRISTOPHER H.",SMITH,CHRISTOPHER,H,"Sen. Murphy, Christopher [D-CT]*",MURPHY,CHRISTOPHER
2,-1000.0,"MALONEY, SEAN PATRICK",MALONEY,SEAN,H,"Sen. Murray, Patricia [D-WA]",MURRAY,PATRICIA
3,-1000.0,"MALONEY, SEAN PATRICK",MALONEY,SEAN,H,"Sen. Leahy, Patrick J. [D-VT]*",LEAHY,PATRICK
4,1000.0,"LEAHY, PATRICK JOSEPH",LEAHY,PATRICK,S,"Sen. Leahy, Patrick J. [D-VT]*",LEAHY,PATRICK
5,1000.0,"MALONEY, SEAN PATRICK",MALONEY,SEAN,H,"Sen. Murray, Patricia [D-WA]",MURRAY,PATRICIA
6,1000.0,"MALONEY, SEAN PATRICK",MALONEY,SEAN,H,"Sen. Leahy, Patrick J. [D-VT]*",LEAHY,PATRICK
7,1000.0,"MALONEY, SEAN PATRICK",MALONEY,SEAN,H,"Sen. Murray, Patricia [D-WA]",MURRAY,PATRICIA
8,1000.0,"MALONEY, SEAN PATRICK",MALONEY,SEAN,H,"Sen. Leahy, Patrick J. [D-VT]*",LEAHY,PATRICK
9,1000.0,"HASSAN, MARGARET WOOD",HASSAN,MARGARET,S,"Sen. Hassan, Margaret Wood [D-NH]*",HASSAN,MARGARET
